# **Research Question: Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018**

**1.0 Importing the Libraries to be used**

In [1]:
# We will be utilizating the numpy and pandas libraries in this notebook in the
# prearation and analysis of the Autolib data, the we need to import the two libraries
# into our programming environment.
import numpy as np # will import numpy library
import pandas as pd # to import pandas library

**1.1 Reading the Dataset from our CSV file**

Our data will be drawn from the below data file:

Autolib_dataset.xlsx [http://bit.ly/autolib_dataset]




In [2]:
# After importing the libraries that we will use, the next step is to upload our datasets to our environment
# The we will load it using pandas and preview it to get a clear picture of the type of date we will be dealing 
# with and guide our next processes of its cleaning and later analysis.
Autolib = pd.read_csv('Autolib_dataset.csv')

In [ ]:
# Lets now preview our data.
Autolib.head(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


**1.2 Accessing Information about our Dataset**

In [ ]:
# After previewing our data, lets now get to know more about our dataset by accessing its information
Autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

**1.3 Data Cleaning**

We will drop the below columns that we will not need in our Analysis.

Displayed comment,

Scheduled at

Geo point

Cars (Since it is basically a duplicate of the bluecars counter, it can be dropped)

Charge Slots

Charging Status

ID

Rental

Slots

year

month

minute

In [4]:
# Data cleaning: Removing Irrelevant Data Observation
# We will drop columns that are irrevant
# First we will drop the Displayed comment column since no question requires us to  us that data
Autolib.drop('Displayed comment', axis='columns', inplace=True)

In [5]:
# Next we will drop the scheduled at column which usually contaims null values and is 
# not required to solve our problem.
Autolib.drop('Scheduled at', axis='columns', inplace=True)

In [16]:
# Let drop the other remaining columns at once
Autolib.drop(['Charge Slots','Slots','Charging Status','Rental status','year','month','minute','Cars','Geo point','ID'],axis=1, inplace=True)

In [17]:
# Lets now access our datset again.
Autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Bluecar counter      5000 non-null   int64 
 2   Utilib counter       5000 non-null   int64 
 3   Utilib 1.4 counter   5000 non-null   int64 
 4   City                 5000 non-null   object
 5   Kind                 5000 non-null   object
 6   Postal code          5000 non-null   int64 
 7   Public name          5000 non-null   object
 8   Station type         5000 non-null   object
 9   Status               5000 non-null   object
 10  Subscription status  5000 non-null   object
 11  day                  5000 non-null   int64 
 12  hour                 5000 non-null   int64 
dtypes: int64(6), object(7)
memory usage: 507.9+ KB


In [ ]:
# Next step in the cleaning process is to check whether there are any missing data
# or null values
np.count_nonzero(Autolib.isnull())

# Our data has no missing values

0

In [19]:
# Then we detect and drop any duplicates in our data
Autolib[Autolib.duplicated()].head()

Autolib = Autolib.drop_duplicates()

# Let check the number of rows and columns after dropping the duplicates.
Autolib.shape

(4968, 13)

**1.4 Detecting Outliers**

In [ ]:
# Lets now see whether our data has any outliers 
# We will Z-score to do this
outliers=[]

def detect_outliers(Autolib):
    
    threshold=3
    mean_1 = np.mean(Autolib)
    std_1 =np.std(Autolib)
    
    for y in Autolib:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold:
            outliers.append(y)
    return outliers
 
detect_outliers(Autolib['day'])
detect_outliers(Autolib['hour'])
detect_outliers(Autolib['minute'])  

# Our data has no ouliers in the three key columns required to answer our reserch question.

[]

In [21]:
# To make the data more easy to query we will change the case of column names to lower, 
# stripp whitespaces, remove periods and replace spaces with underscore
Autolib.columns = Autolib.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.', '')

Autolib.columns

Index(['address', 'bluecar_counter', 'utilib_counter', 'utilib_14_counter',
       'city', 'kind', 'postal_code', 'public_name', 'station_type', 'status',
       'subscription_status', 'day', 'hour'],
      dtype='object')

**1.5 Answering Questions**



In [ ]:
# First we will need to adda new columns to our dataframe that shows the usage of all the three types
# of electric cars
#Creating column for Bluecar usage
Autolib['bluecar_usage'] = Autolib.bluecar_counter.diff()

#Creating column for utilib usage
Autolib['utilib_usage'] = Autolib.utilib_counter.diff()

#Creating column for utilib 1.4 usage
Autolib['utilib_14_usage'] = Autolib.utilib_14_counter.diff()


In [25]:
# Lets preview our new dataframe now.
Autolib.head()

# From the dataframe +ve number indicates that a car was returned -ve number indicates that a car was picked 
# 0 means that nothing happened, that is, no car was picked nor returned

,address,bluecar_counter,utilib_counter,utilib_14_counter,city,kind,postal_code,public_name,station_type,status,subscription_status,day,hour,bluecar_usage,utilib_usage,utilib_14_usage
0,2 Avenue de Suffren,0,0,0,Paris,STATION,75015,Paris/Suffren/2,station,ok,nonexistent,8,11,NaN,NaN,NaN
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,station,ok,nonexistent,6,7,6.0,0.0,0.0
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,station,ok,nonexistent,3,20,-3.0,0.0,2.0
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,station,ok,nonexistent,4,4,0.0,1.0,-2.0
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,station,ok,nonexistent,8,17,0.0,-1.0,0.0


**Question one: Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.**

In [26]:
# Now that we have finished cleaning our data and we have no missing data
# or outliers, we can proceed and answer our reserch question.
# We will start with our main question and then move the other questions
# Lets first see the unique values in blucar_usage column
Autolib.bluecar_usage.unique()



array([nan,  6., -3.,  0.,  3., -2.,  5., -6.,  1.,  4., -5., -1.,  2.,
       -4.,  7., -7.])

In [27]:
# Then we see the hour of the day which was most popular.
Autolib[(Autolib.city == 'Paris') & (Autolib.bluecar_usage == -7.0)].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

# The most popular hour of the day for picking cars is when the difference
# is at maximum (-7) which is 1400 hours (or 2p.m.)

hour
14    1
Name: hour, dtype: int64

**Question 2: What is the most popular hour for returning cars?**

In [54]:
# Lets now find out the most popular hour fro returning cars.
Autolib[Autolib.bluecar_usage == 7.].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

# The most popular hour for returning cars is when the difference is at the highest possible positive difference (7.0)
# which is at 0000 hours (or 12a.m.)

hour
0    4
Name: hour, dtype: int64

**Question 3(a). What station is the most popular overall?**

In [29]:
# Here we will find the most popular station for the month of April
Autolib[(Autolib.kind == 'STATION') & (Autolib.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

# The most popular station overall is Paris/Porte de Montrouge/8

,kind
public_name,
Paris/Porte de Montrouge/8,13


**Question 3(b). What station is the most popular At the most popular picking hour?**

In [30]:
# Now we get the popula station at the most popular picking hour
Autolib[(Autolib.kind == 'STATION') & (Autolib.hour == 14)  & (Autolib.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

# The most popular station at the most popular picking hour is Paris/Aumale/28

,kind
public_name,
Paris/Aumale/28,3


**Question 4(a). What postal code is the most popular for picking up Blue cars overal? Does the most popular station belong to that postal code?**

In [31]:
# Now we find out the most popular postal code for picking bluecars overall.
Autolib[Autolib.bluecar_usage < 0].groupby('postal_code').count()[['bluecar_usage']].sort_values(by = 'bluecar_usage', ascending = False).head(1)

# The most popular postal code is 75015


,bluecar_usage
postal_code,
75015,113


In [32]:
# The second part of the question is to find out if the most popular ststion belong to the above postal code.
Autolib[Autolib.public_name == 'Paris/Porte de Montrouge/8'][['public_name', 'postal_code']].head(1)

# The most popular station does not belong to the most popular psotal code.

,public_name,postal_code
122,Paris/Porte de Montrouge/8,75014


**Question 4(b). What postal code is the most popular for picking up Blue cars At the most popular picking hour? Does the most popular station belong to that postal code?**

In [33]:
# Next we find out the most popular postal code at the most popular hour
Autolib[(Autolib.bluecar_usage < 0) & (Autolib.hour == 14)].groupby('postal_code').count()[['bluecar_usage']].sort_values(by = 'bluecar_usage', ascending = False).head(1)

# The most popular postal code at the most popular hour is 75010

,bluecar_usage
postal_code,
75010,6


In [34]:
# The second part of the question is to find out if the most popular station
# At the most popular hour belong to the above postal code.
Autolib[(Autolib.public_name == 'Paris/Aumale/28') & (Autolib.hour == 14)][['public_name', 'postal_code']].head(1)

# The most popular station at the most popular hour does not belong to the most popular postal code.

,public_name,postal_code
2450,Paris/Aumale/28,75009


**Question 5. Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars?**

**(a). Utilib**

In [35]:
# Unique values to show us the range between the mot negative and most poitive
Autolib.utilib_usage.unique()

# The maximum negative number (-2) shows when the picking of the cars was highest.
# Similly, the maximum positive number (2) shows when the returning of the cars was highest

array([nan,  0.,  1., -1.,  2., -2.])

In [56]:
# Most popular hour of the day for picking up a utilib car in the city of Paris over the month of April 2018
Autolib[(Autolib.city == 'Paris') & (Autolib.utilib_usage == -2 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)


# The most popular hour for picking utilib cars is 1000Hrs or 10am which is different from the one for Bluecars

hour
10    2
Name: hour, dtype: int64

In [45]:
# Most popular hour for returning utilib cars
Autolib[(Autolib.city == 'Paris') & (Autolib.utilib_usage == 2 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

# The most popular hour for returning utilib cars is 0800Hrs or 8am which is different from the one for Bluecars

hour
8    3
Name: hour, dtype: int64

In [39]:
# Utilib most popular station overall
Autolib[(Autolib.kind == 'STATION') & (Autolib.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

# The most popular station does not change.

,kind
public_name,
Paris/Porte de Montrouge/8,13


In [40]:
# Utilib most popular station at the most popular picking hour
Autolib[(Autolib.kind == 'STATION') & (Autolib.hour == 10)  & (Autolib.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

# The most popular station at the most popular picking hour does is different from the one for Bluacars.

,kind
public_name,
SÃ¨vres/WolfenbÃ¼ttel/1,3


In [41]:
# Most popular postal code when picking up Utilib cars
Autolib[Autolib.utilib_usage < 0].groupby('postal_code').count()[['utilib_usage']].sort_values(by = 'utilib_usage', ascending = False).head(1)

# The most popular postal code is the same as the obe for Bluecars

,utilib_usage
postal_code,
75015,21


In [42]:
# Does the most popular station belong to that postal code 
Autolib[Autolib.public_name == 'Paris/Porte de Montrouge/8'][['public_name', 'postal_code']].head(1)

# The most popular station does not belong that postal code.

,public_name,postal_code
122,Paris/Porte de Montrouge/8,75014


**(b). Utilib 1.4**

In [43]:
# Unique values to show us the range between the mot negative and most poitive
Autolib.utilib_14_usage.unique()

# The maximum negative number (-3) shows when the picking of the cars was highest.
# Similly, the maximum positive number (3) shows when the returning of the cars was highest

array([nan,  0.,  2., -2.,  1., -1.,  3., -3.])

In [44]:
# Most popular hour of the day for picking up a utilib 1.4 car in the city of Paris over the month of April 2018
Autolib[(Autolib.city == 'Paris') & (Autolib.utilib_14_usage == -3 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

# The most popular hour for picking utilib 1.4 cars is 0200Hrs or 2am which is different from the one for Bluecars

hour
2    1
Name: hour, dtype: int64

In [46]:
# Most popular hour for returning utilib 1.4 cars
Autolib[(Autolib.city == 'Paris') & (Autolib.utilib_14_usage == 3 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

# The most popular hour for returning utilib 1.4 cars is 1300Hrs or 1pm which is different from the one for Bluecars

hour
13    1
Name: hour, dtype: int64

In [47]:
# Utilib 1.4 most popular station overall
Autolib[(Autolib.kind == 'STATION') & (Autolib.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

# The most popular station does not change.

,kind
public_name,
Paris/Porte de Montrouge/8,13


In [48]:
# Utilib 1.4 most popular station at the most popular picking hour
Autolib[(Autolib.kind == 'STATION') & (Autolib.hour == 2)  & (Autolib.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

# The most popular station at the most popular picking hour is different from the one for Bluacars.

,kind
public_name,
Saint-Ouen/Louis Blanc/35,3


In [50]:
# Most popular postal code when picking up Utilib cars
Autolib[Autolib.utilib_usage < 0].groupby('postal_code').count()[['utilib_14_usage']].sort_values(by = 'utilib_14_usage', ascending = False).head(1)

# The most popular postal code is the same as the obe for Bluecars

,utilib_14_usage
postal_code,
75015,21


In [51]:
# Does the most popular station belong to that postal code 
Autolib[Autolib.public_name == 'Saint-Ouen/Louis Blanc/35'][['public_name', 'postal_code']].head(1)

# The most popular station does not belong that postal code.

,public_name,postal_code
1163,Saint-Ouen/Louis Blanc/35,93400
